In [ ]:
import numpy as np
import pandas as pd
import pickle

from matplotlib import pyplot as plt

import plotly.graph_objects as go
import plotly.express as px

In [ ]:
%cd /Users/alexandrine/Library/CloudStorage/OneDrive-TechnischeUniversitätBerlin/causal_detection_of_CSD/
import functions.data_analysis as analysis

/Users/alexandrine/Library/CloudStorage/OneDrive-TechnischeUniversitätBerlin/causal_detection_of_CSD


In [4]:
file_paths = [
    "/Users/alexandrine/Library/CloudStorage/OneDrive-TechnischeUniversitätBerlin/causal_detection_of_CSD/results_causalCSD/saddle-node/methods/Apr25/causalEE/falseAlarm-complex-0.01-KNeighborsRegressor-rmvDet-StandardScaler-1000-1-100.dat",
    "/Users/alexandrine/Library/CloudStorage/OneDrive-TechnischeUniversitätBerlin/causal_detection_of_CSD/results_causalCSD/saddle-node/methods/Apr25/causalEE/falseAlarm-complex-0.01-KNeighborsRegressor-rmvDet-StandardScaler-1000-100-200.dat",
]

In [5]:
merged_data = {}

for i, path in enumerate(file_paths):
    with open(path, "rb") as f:
        data = pickle.load(f)
    
    if i == 0:
        merged_data = {k: list(v) for k, v in data.items()}
    else:
        for k in merged_data:
            merged_data[k].extend(data[k])

output_path = "/Users/alexandrine/Library/CloudStorage/OneDrive-TechnischeUniversitätBerlin/causal_detection_of_CSD/results_causalCSD/saddle-node/methods/Apr25/causalEE/falseAlarm-complex-0.01-KNeighborsRegressor-rmvDet-1000.dat"

with open(output_path, "wb") as f:
    pickle.dump(merged_data, f)

print(f"Fichier concaténé sauvegardé à : {output_path}")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/alexandrine/Library/CloudStorage/OneDrive-TechnischeUniversitätBerlin/causal_detection_of_CSD/results_causalCSD/saddle-node/methods/Apr25/causalEE/falseAlarm-complex-0.01-KNeighborsRegressor-rmvDet-StandardScaler-1000-1-100.dat'

In [ ]:
# Chemin du fichier concaténé
file_path = "/Users/alexandrine/Library/CloudStorage/OneDrive-TechnischeUniversitätBerlin/causal_detection_of_CSD/results_causalCSD/saddle-node/methods/Apr25/causalEE/falseAlarm-complex-0.01-KNeighborsRegressor-rmvDet" \
"-1000.dat"

# Chargement du dictionnaire
with open(file_path, "rb") as f:
    data = pickle.load(f)

# Vérification des longueurs
print("Longueur des entrées dans le fichier concaténé :")
for k, v in data.items():
    try:
        print(f"{k}: {len(v)} éléments")
    except TypeError:
        print(f"{k}: non itérable (type {type(v)})")

Longueur des entrées dans le fichier concaténé :
data: 200 éléments
truncated_data: 200 éléments
indicator: 200 éléments
residuals: 200 éléments
pvalue: 200 éléments
computation_time: 200 éléments


    residuals = y - (slope * x + intercept)
    mean_residual = np.mean(np.abs(residuals))

In [5]:
mypath = '/Users/alexandrine/Library/CloudStorage/OneDrive-TechnischeUniversitätBerlin/causal_detection_of_CSD/results_causalCSD/saddle-node/methods/Apr25/causalEE/'

In [6]:
#load data
base_data_dict = analysis.load_and_rename_files(mypath, 'base-*.dat')
confounderDecreasAC_data_dict = analysis.load_and_rename_files(mypath, 'confounderDecreasAC-*.dat')
confounderIncreasAC_data_dict = analysis.load_and_rename_files(mypath, 'confounderIncreasAC-*.dat')
falseAlarm_data_dict = analysis.load_and_rename_files(mypath, 'falseAlarm-*.dat')

In [8]:
# 200 realizations each (150 for falseAlarm)
# base_data_dict = analysis.truncate_dict_values(base_data_dict)
# confounderDecreasAC_data_dict = analysis.truncate_dict_values(confounderDecreasAC_data_dict)
# confounderIncreasAC_data_dict = analysis.truncate_dict_values(confounderIncreasAC_data_dict)
# falseAlarm_data_dict = analysis.truncate_dict_values(falseAlarm_data_dict)

In [9]:
len(falseAlarm_data_dict['falseAlarm_linear_001_KNeighborsRegressor_rmvDet_1000']['residuals'])

200

In [7]:
all_data_dicts = {
    'base': base_data_dict,
    'confounderDecreasAC': confounderDecreasAC_data_dict,
    'confounderIncreasAC': confounderIncreasAC_data_dict,
    'falseAlarm': falseAlarm_data_dict
}

In [8]:
models = ['base', 
          'confounderDecreasAC', #'confounderIncreasAC',
         'falseAlarm'
         ]
X_functions = ['linear', 
               'square', #'complex'
               ]
sigma_Y = "001"
methods=['KNeighborsRegressor', 'LinearRegression', 
         #'regular', 'bb', 'rosa'
         ]
window_length = "1000"
detrend='rmvDet'

In [9]:
def aggregate_residuals_causalEE(method='KNeighborsRegressor', window_length=1000):
    residuals_agg_dict = {}
    #std_devs_agg_dict = {}
    #ratio_agg_dict = {}


    for i, X_function in enumerate(X_functions, start=1):
        for j, model in enumerate(models, start=1):
            model_data_dict = all_data_dicts[model]
            var_name = f"{model}_{X_function}_{sigma_Y}_{method}_{detrend}_{window_length}"

            residuals_agg_dict[f"{model}_{X_function}"] = []
            #std_devs_agg_dict[f"{model}_{X_function}"] = []
            #ratio_agg_dict[f"{model}_{X_function}"] = []

            for k in range(len(model_data_dict[var_name]['residuals'])):
                residuals_agg_dict[f"{model}_{X_function}"].append(model_data_dict[var_name]['residuals'][k])
                #std_devs_agg_dict[f"{model}_{X_function}"].append(model_data_dict[var_name]['std_devs'][k])
                #ratio_agg_dict[f"{model}_{X_function}"].append([r/sd for r, sd in zip(model_data_dict[var_name]['residuals'][k], model_data_dict[var_name]['std_devs'][k])])

    return residuals_agg_dict

In [10]:
models = ['base', 
          'confounderDecreasAC', #'confounderIncreasAC', 
          'falseAlarm'
          ]

X_functions = ['linear', 
    #'square', 
    #'complex'
               ]

In [11]:
residuals_agg_dict = aggregate_residuals_causalEE(method='KNeighborsRegressor')

In [12]:
def plot_residuals_boxplot_causalEE(agg_dict, method='KNeighborsRegressor'):

    data = []

    for key, value in agg_dict.items():
        residuals = np.concatenate(value) # residuals for each simulation are concatenated into one array
        data.extend([(key, r) for r in residuals])
    
    df = pd.DataFrame(data, columns=["Key", "Residuals"])

    fig = px.box(
        df,
        x="Key",
        y="Residuals",
        points="outliers",
        title=f"{method} - Residuals Box Plot",
    )
    fig.update_traces(marker_color='#AFDDE9', line_color = '#37ABC8', boxmean=False)
    fig.update_layout(width=800, height=600,
        xaxis_title="Scenario",
        yaxis_title="Residuals",
        plot_bgcolor='rgba(215, 238, 244, 0.3)'
        #template="plotly_white",
    )
    return fig


In [15]:
def plot_residuals_boxplot_causalEE(agg_dict, method='KNeighborsRegressor'):
    """
    Boxplot léger : une trace box par scénario (sans points) + une seule trace scatter
    pour tous les points, coloriés par l'index temporel 'Window'.
    """
    # Construire le DataFrame avec colonne Window (index temporel croissant)
    data = []
    for key, list_of_arrays in agg_dict.items():
        window_counter = 0
        for arr in list_of_arrays:
            for r in np.asarray(arr).ravel():
                data.append((key, r, window_counter))
                window_counter += 1
    df = pd.DataFrame(data, columns=["Key", "Residuals", "Window"])

    # Ordre des catégories et positions numériques correspondantes
    keys = list(df["Key"].unique())
    x_positions = {k: i for i, k in enumerate(keys)}

    # Figure : ajouter une Box par catégorie (sans points)
    fig = go.Figure()
    for key in keys:
        sub = df[df["Key"] == key]
        fig.add_trace(
            go.Box(
                y=sub["Residuals"],
                x=[key] * len(sub),     # garde l'étiquette catégorielle pour l'axe x
                name=key,
                boxpoints=False,        # PAS de points ici
                whiskerwidth=0.5,
                marker=dict(line=dict(width=1)),
                showlegend=False,
            )
        )

    # Préparer positions x numériques + jitter pour la trace scatter (une seule trace)
    # Pour garder les ticks catégoriels, on utilisera les positions numériques + jitter,
    # puis on fixera les ticks de l'axe x aux noms de catégories.
    rng = np.random.RandomState(0)
    x_numeric = []
    y_all = df["Residuals"].values
    color_all = df["Window"].values
    for k in df["Key"].values:
        base = x_positions[k]
        # jitter faible (ajustable) pour mieux visualiser la densité
        x_numeric.append(base + rng.uniform(-0.25, 0.25))

    # Trace unique de tous les points, colorée par 'Window'
    fig.add_trace(
        go.Scatter(
            x=x_numeric,
            y=y_all,
            mode='markers',
            marker=dict(
                color=color_all,
                colorscale='Viridis',
                cmin=df["Window"].min(),
                cmax=df["Window"].max(),
                size=6,
                opacity=0.75,
                colorbar=dict(title="Window"),
            ),
            hovertemplate="Scenario: %{customdata[0]}<br>Residual: %{y:.4f}<br>Window: %{marker.color}<extra></extra>",
            customdata=np.stack([df["Key"].values], axis=1),
            showlegend=False,
        )
    )

    # Réglages d'axes : remplacer ticks numériques par catégories
    fig.update_xaxes(
        tickmode='array',
        tickvals=list(x_positions.values()),
        ticktext=keys,
    )

    fig.update_layout(
        title=f"{method} - Residuals Box Plot (échelle temporelle)",
        width=900,
        height=650,
        xaxis_title="Scenario",
        yaxis_title="Residuals",
        plot_bgcolor='rgba(215, 238, 244, 0.3)',
    )

    return fig


In [15]:
def plot_residuals_boxplot_causalEE(agg_dict, method='KNeighborsRegressor', bin_size=500, jitter=0.18):
    """
    Boxplot + points groupés par tranches temporelles.
    - bin_size : taille des tranches (ex. 500)
    - jitter : amplitude du jitter horizontal
    Chaque point est tracé une seule fois. On crée une trace scatter par tranche.
    """
    import numpy as np
    import pandas as pd
    import plotly.graph_objects as go
    import plotly.express as px

    # Construire le DataFrame (Key, Residuals, Window)
    data = []
    for key, list_of_arrays in agg_dict.items():
        window_counter = 0
        for arr in list_of_arrays:
            a = np.asarray(arr).ravel()
            n = a.size
            # on évite boucle point par point en étendant en bloc
            data.extend([(key, float(v), w) for v, w in zip(a, range(window_counter, window_counter + n))])
            window_counter += n
    df = pd.DataFrame(data, columns=["Key", "Residuals", "Window"])

    if df.empty:
        raise ValueError("agg_dict produit un DataFrame vide.")

    # Catégories et positions numériques
    keys = list(df["Key"].unique())
    x_positions = {k: i for i, k in enumerate(keys)}

    # Figure et boîtes (sans points)
    fig = go.Figure()
    for key in keys:
        sub = df[df["Key"] == key]
        fig.add_trace(
            go.Box(
                y=sub["Residuals"].values,
                x=[key] * len(sub),
                name=key,
                boxpoints=False,
                whiskerwidth=0.5,
                marker=dict(line=dict(width=1)),
                showlegend=False,
            )
        )

    # Définir bins (groupes) sur Window
    max_w = int(df["Window"].max())
    bins = np.arange(0, max_w + bin_size, bin_size)
    # np.digitize renvoie l'indice du bin (1..len(bins)), on le ramène à 0-based
    bin_idx = np.digitize(df["Window"].values, bins, right=False) - 1
    df["BinIdx"] = bin_idx

    # Palette discrète: échantillon de la palette séquentielle adaptée au nombre de bins
    unique_bins = np.unique(bin_idx)
    n_bins = unique_bins.size
    palette = px.colors.sample_colorscale("Viridis", [i/(max(1, n_bins-1)) for i in range(n_bins)])
    # fallback si palette trop courte (rare)
    if len(palette) < n_bins:
        palette = px.colors.qualitative.Plotly * ((n_bins // len(px.colors.qualitative.Plotly)) + 1)
        palette = palette[:n_bins]

    # Positions numériques de base et jitter vectorisé (reproductible)
    rng = np.random.RandomState(0)
    bases = np.array([x_positions[k] for k in df["Key"].values])
    jitter_vec = rng.uniform(-jitter, jitter, size=len(df))
    x_numeric_all = bases + jitter_vec

    # Trace unique par bin (contient points de toutes les catégories appartenant à ce bin)
    for i, b in enumerate(unique_bins):
        mask = df["BinIdx"].values == b
        if not np.any(mask):
            continue
        xi = x_numeric_all[mask]
        yi = df["Residuals"].values[mask]
        windows_i = df["Window"].values[mask]
        keys_i = df["Key"].values[mask]

        fig.add_trace(
            go.Scatter(
                x=xi,
                y=yi,
                mode='markers',
                marker=dict(
                    color=palette[i],
                    size=6,
                    opacity=0.8,
                ),
                name=f"{int(b*bin_size)}–{int(min((b+1)*bin_size-1, max_w))}",  # label lisible pour la légende
                hovertemplate="Scenario: %{customdata[0]}<br>Residual: %{y:.4f}<br>Window: %{customdata[1]}<extra></extra>",
                customdata=np.stack([keys_i, windows_i], axis=1),
                showlegend=True,
            )
        )

    # Remplacer ticks numériques par catégories
    fig.update_xaxes(
        tickmode='array',
        tickvals=list(x_positions.values()),
        ticktext=keys,
    )

    fig.update_layout(
        title=f"{method} - Residuals Box Plot (bins de {bin_size} fenêtres)",
        width=900,
        height=650,
        xaxis_title="Scenario",
        yaxis_title="Residuals",
        plot_bgcolor='rgba(215, 238, 244, 0.3)',
        legend_title="Window groups",
    )

    return fig

In [16]:
fig = plot_residuals_boxplot_causalEE(residuals_agg_dict, method='KNeighborsRegressor')

In [17]:
fig.write_image("residuals_boxplot_static.png", width=1200, height=800, scale=2)

KeyboardInterrupt: 

In [ ]:
fig.show()

In [ ]:
def plot_residuals_matplotlib_causalEE2(agg_dict, method='KNeighborsRegressor'):
    # Rassembler les données dans un DataFrame
    data = []
    for key, value in agg_dict.items():
        residuals = np.concatenate(value)
        data.append(residuals)

    # Créer le box plot
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.boxplot(data, labels=agg_dict.keys(), showmeans=True, patch_artist=True, 
               boxprops=dict(facecolor='lightblue', color='blue'),
               meanprops=dict(marker='o', markerfacecolor='red', #markeredgecolor='black'
                              ),
               flierprops=dict(marker='x', color='blue', alpha=0.1)
               )

    # Ajouter les titres et légendes
    ax.set_title(f"{method} - Residuals Box Plot", fontsize=14)
    #ax.set_xlabel("Key", fontsize=12)
    ax.set_ylabel("Residuals (absolute value) / Standard deviations", fontsize=12)

    # Personnaliser l'apparence
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()

    return fig, ax


In [ ]:
fig, ax = plot_residuals_matplotlib_causalEE2(residuals_agg_dict, method='KNeighborsRegressor')
fig.savefig("residuals_boxplot_matplotlib.png", dpi=300)
plt.show()

In [ ]:
#fig = plot_residuals_causalEE2(residuals_agg_dict, method='causalKNN')
#fig.write_image("plots/residuals_boxplot2.png", width=800, height=600)

In [ ]:
#fig.show()